# Guardian - Iteration 8

## Import Packages

In [13]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [14]:
iteration = 8

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "show-fields": 
        """headline,
        firstPublicationDate,
        productionOffice, 
        bodyText, 
        charCount, 
        commentable, 
        starRating, 
        isLive"""
    ,
    "show-elements": "all",
    "show-tags": ["all"],
    "show-rights": "syndicatable",
    "from-date": "2019-02-01",
    "to-date": "2019-07-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY3"),
}


In [15]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [16]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [17]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=headline%2C%0A++++++++firstPublicationDate%2C%0A++++++++productionOffice%2C+%0A++++++++bodyText%2C+%0A++++++++charCount%2C+%0A++++++++commentable%2C+%0A++++++++starRating%2C+%0A++++++++isLive&show-elements=all&show-tags=all&show-rights=syndicatable&from-date=2019-02-01&to-date=2019-07-01&order-by=newest&api-key=7e9dff14-114c-40a2-bb71-a816ac25ee64&page=1
status ok
total 33006
startIndex 1
pageSize 50
pages 661
orderBy newest
---- RUNTIME STATUS ---- 
(1/661) in 0:00:00.620700s
(2/661) in 0:00:00.356610s
(3/661) in 0:00:00.373289s
(4/661) in 0:00:00.637824s
(5/661) in 0:00:00.398049s
(6/661) in 0:00:00.696086s
(7/661) in 0:00:00.357240s
(8/661) in 0:00:00.659488s
(9/661) in 0:00:00.682991s
(10/661) in 0:00:00.421147s
(11/661) in 0:00:00.634180s
(12/661) in 0:00:00.454987s
(13/661) in 0:00:00.598474s
(14/661) in 0:00:00.554065s
(15/661) in 0:00:00.479606s
(16/661) in 0:00:00.503821s
(17

In [18]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields", "rights"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}.csv.zip", compression="zip")
    return df

In [19]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(33006, 14) took 0:00:00.009398s

unfold_columns:
 shape=(33006, 17) took 0:00:01.266194s

create_csv:
 shape=(33006, 17) took 0:00:02.979670s

